# Recommender Systems - notebook 1 - Traditional Approaches


In [1]:
"""
(Practical tip) Table of contents can be compiled directly in jupyter notebooks using the following code:
I set an exception: if the package is in your installation you can import it otherwise you download it 
then import it.
"""
try:
    from jyquickhelper import add_notebook_menu 
except:
    !pip install jyquickhelper
    from jyquickhelper import add_notebook_menu
    
"""
Output Table of contents to navigate easily in the notebook. 
For interested readers, the package also includes Ipython magic commands to go back to this cell
wherever you are in the notebook to look for cells faster
"""
add_notebook_menu()

## Imports


In [2]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context


In [3]:
from tqdm import tqdm
import pandas as pd
import numpy as np


## Dataset description


We use here the MovieLens 100K Dataset. It contain 100,000 ratings from 1000 users on 1700 movies.

- u.train / u.test part of the original u.data information
  - The full u data set, 100000 ratings by 943 users on 1682 items.
    Each user has rated at least 20 movies. Users and items are
    numbered consecutively from 1. The data is randomly
    ordered. This is a tab separated list of
    user id | item id | rating | timestamp.
    The time stamps are unix seconds since 1/1/1970 UTC
- u.info
  - The number of users, items, and ratings in the u data set.
- u.item
  - Information about the items (movies); this is a tab separated
    list of
    movie id | movie title | release date | video release date |
    IMDb URL | unknown | Action | Adventure | Animation |
    Children's | Comedy | Crime | Documentary | Drama | Fantasy |
    Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi |
    Thriller | War | Western |
    The last 19 fields are the genres, a 1 indicates the movie
    is of that genre, a 0 indicates it is not; movies can be in
    several genres at once.
    The movie ids are the ones used in the u.data data set.
- u.genre
  - A list of the genres.
- u.user
  - Demographic information about the users; this is a tab
    separated list of
    user id | age | gender | occupation | zip code
    The user ids are the ones used in the u.data data set.


In [5]:
path = "https://www.i3s.unice.fr/~riveill/dataset/dataset_movilens_100K/"


Before we build our model, it is important to understand the distinction between implicit and explicit feedback, and why modern recommender systems are built on implicit feedback.

- **Explicit Feedback:** in the context of recommender systems, explicit feedback are direct and quantitative data collected from users.
- **Implicit Feedback:** on the other hand, implicit feedback are collected indirectly from user interactions, and they act as a proxy for user preference.

The advantage of implicit feedback is that it is abundant. Recommender systems built using implicit feedback allow recommendations to be adapted in real time, with each click and interaction.

Today, online recommender systems are built using implicit feedback.


### Data preprocessing


In [6]:
# Load data
np.random.seed(123)

ratings = pd.read_csv(
    path + "u.data",
    sep="\t",
    header=None,
    names=["userId", "movieId", "rating", "timestamp"],
)
ratings = ratings.sort_values(["timestamp"], ascending=True)
print("Nb ratings:", len(ratings))
ratings.head()


Nb ratings: 100000


,userId,movieId,rating,timestamp
214,259,255,4,874724710
83965,259,286,4,874724727
43027,259,298,4,874724754
21396,259,185,4,874724781
82655,259,173,4,874724843


### Data splitting

Separating the dataset between train and test in a random fashion would not be fair, as we could potentially use a user's recent evaluations for training and previous evaluations. This introduces a data leakage with an anticipation bias, and the performance of the trained model would not be generalizable to real world performance.

Therefore, we need to slice the train and test based on the timestamp


In [12]:
# Split dataset
train_ratings, test_ratings = np.split(ratings, [int(0.9 * len(ratings))])

max(train_ratings["timestamp"]) <= min(test_ratings["timestamp"])


True

In [13]:
# drop columns that we no longer need
train_ratings = train_ratings[["userId", "movieId", "rating"]]
test_ratings = test_ratings[["userId", "movieId", "rating"]]

len(train_ratings), len(test_ratings)


(90000, 10000)

In [14]:
# Get a list of all movie IDs
all_movieIds = ratings["movieId"].unique()


### Build pivot table


In [15]:
""" Pivot table for train set """
train_pivot = pd.pivot_table(
    train_ratings, values="rating", index="userId", columns="movieId"
)
print("Nb users: ", train_pivot.shape[0])
print("Nb movies:", train_pivot.shape[1])
display(train_pivot.head())


Nb users:  867
Nb movies: 1637


movieId,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
userId,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,4.0,NaN,NaN,NaN,NaN,NaN,2.0,4.0,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
train_users = train_pivot.index
train_movies = train_pivot.columns


## Collaborative filtering based on Users similarity

This approach uses scores that have been assigned by other users to calculate predictions.

In pivot table

- Rows are users, $u, v$
- Columns are items, $i, j$

$$pred(u, i) = \frac{\sum_v sim(u,v)*r_{v,i}}{\sum_v sim(u,v)}$$

Wich similarity function:

- Euclidean distance $[0,1]$: $sim(a,b)=\frac{1}{1+\sqrt{\sum_i (r_{a,i}-r_{b,i})^2}}$
- Pearson correlation $[-1,1]$: $sim(a,b)=\frac{\sum_i (r_{a,i}-r_a)(r_{b,i}-r_b)}{{\sum_i (r_{a,i}-r_a)^2}{\sum_i (r_{b,i}-r_b)^2}}$
- Cosine similarity $[-1,1]$: $sim(a, b)=\frac{a.b}{|a|.|b|}$

Which function should we use? The answer is that there is no fixed recipe; but there are some issues we can take into account when choosing the proper similarity function. On the one hand:

- Pearson correlation usually works better than Euclidean distance since it is based more on the ranking than on the values. So, two users who usually like more the same set of items, although their rating is on different scales, will come out as similar users with Pearson correlation but not with Euclidean distance.
- On the other hand, when dealing with binary/unary data, i.e., like versus not like or buy versus not buy, instead of scalar or real data like ratings, cosine distance is usually used.


### Build predictor


In [13]:
# Step 1: build the similarity matrix between users
correlation_matrix = train_pivot.transpose().corr("pearson")
correlation_matrix.head()


userId,1,2,3,5,6,7,8,9,10,12,...,934,935,936,937,938,939,940,941,942,943
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.160841,0.112780,0.420809,0.287159,1.237128e-01,0.692086,-0.102062,-0.092344,4.219129e-02,...,0.061695,-0.260242,0.383733,2.899974e-02,0.326744,5.343904e-01,0.263289,0.205616,-0.180784,0.067549
2,0.160841,1.000000,0.067420,0.327327,0.446269,4.807341e-01,0.585491,0.242536,0.668145,3.532708e-16,...,0.021007,-0.271163,0.214017,5.616449e-01,0.331587,-7.671236e-18,-0.011682,-0.062017,0.085960,0.479702
3,0.112780,0.067420,1.000000,NaN,-0.109109,-5.037555e-17,0.291937,NaN,0.311086,1.889822e-01,...,NaN,NaN,-0.045162,-5.233642e-17,-0.137523,NaN,-0.104678,1.000000,-0.011792,NaN
5,0.420809,0.327327,NaN,1.000000,0.241817,1.490373e-01,0.537400,0.577350,0.087343,1.169812e-01,...,0.229532,-0.500000,0.439286,6.085806e-01,0.484211,8.807048e-01,0.027038,0.468521,0.318163,0.346234
6,0.287159,0.446269,-0.109109,0.241817,1.000000,1.758687e-01,0.687745,0.132353,0.273987,1.121937e-01,...,-0.064033,-0.264292,0.392760,2.706660e-01,0.229918,2.063147e-01,-0.024419,0.399186,0.092349,0.109833


In [14]:
# Step2 build rating function
# We want to calculate the rating that a user could have made for an item.

# Il est plus efficace de travailler avec numpy qu'avec pandas.
# On transforme donc la matrice pivot en numpy
pivot = train_pivot.to_numpy()
# idem pour la matrice de correlation
corr = correlation_matrix.to_numpy()
# Malheureusement, on doit utiliser 2 dictionnaires pour passer
# Du nom de la colonne movieId dans son indice en numpy
movie2column = {j: i for i, j in enumerate(train_pivot.columns)}
# Du nom de la ligne userId dans son indice en numpy
user2row = {j: i for i, j in enumerate(train_pivot.index)}


def predict(pivot, corr, userId, movieId):
    if movieId in movie2column.keys():
        movie = movie2column[movieId]
    else:
        return 2.5
    if userId in user2row.keys():
        user = user2row[userId]
    else:
        return 2.5

    # Normalement le rating est inconnu
    if np.isnan(pivot[user, movie]):
        num = 0
        den = 0
        for u in range(len(corr)):
            if not np.isnan(pivot[u, movie]) and not np.isnan(corr[user, u]):
                # Si l'utilisateur u a déjà vu le film movie
                # et si les deux utilisateurs ont au moins vu un même film
                den += abs(corr[user, u])
                num += corr[user, u] * pivot[u, movie]
        if den != 0:
            return num / den
        else:
            return 2.5  # default value
    else:
        # le film a déjà été vu
        print(
            "l'utilisateur",
            userId,
            "a déjà vu le film",
            movieId,
            "et lui a donné la note de",
            pivot[user, movie],
        )
        return pivot[user, movie]


predict(pivot, corr, 1, 1)
predict(pivot, corr, 3, 28)


l'utilisateur 1 a déjà vu le film 1 et lui a donné la note de 5.0


1.8527972301545377

### Predict


In [15]:
# Step 3 add the predicted rating to the test set
pred = []
for _, userId, movieId, _ in tqdm(
    test_ratings[["userId", "movieId", "rating"]].itertuples()
):
    pred += [predict(pivot, corr, userId, movieId)]

test_ratings["User based"] = pred
test_ratings.head()


10000it [00:06, 1592.65it/s]


,userId,movieId,rating,User based
557,90,900,4,0.657995
6675,90,269,5,0.987827
562,90,289,3,0.572064
62660,90,270,4,1.077609
68756,90,268,4,1.202310


### Evaluate the predictor

Now that we have trained our model, assigned a value to each pair (userId, movieId), we are ready to evaluate it.


#### Evaluation with classical metrics: RMSE

In traditional machine learning projects, we evaluate our models using measures such as accuracy (for classification problems) and RMSE (for regression problems). This is what we will do in the first instance.


In [16]:
# Step 4 evaluate the resulte : with classical metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error

print(
    "RMSE:",
    np.sqrt(mean_squared_error(test_ratings["rating"], test_ratings["User based"])),
)


RMSE: 1.7798966446725806


#### Hit Ratio @ K

However, a measure such as RMSE does not provide a satisfactory evaluation of recommender systems. To design a good metric for evaluating recommender systems, we need to first understand how modern recommender systems are used.

Amazon, Netflix and others uses a list of recommendations. The key here is that we don’t need the user to interact with every single item in the list of recommendations. Instead, we just need the user to interact with at least one item on the list — as long as the user does that, the recommendations have worked.

To simulate this, let’s run the following evaluation protocol to generate a list of top 10 recommended items for each user.

- For each user, randomly select 99 items that the user has not interacted with.
- Combine these 99 items with the test item (the actual item that the user last interacted with). We now have 100 items.
- Run the model on these 100 items, and rank them according to their predicted probabilities.
- Select the top 10 items from the list of 100 items. If the test item is present within the top 10 items, then we say that this is a hit.
- Repeat the process for all users. The Hit Ratio is then the average hits.

This evaluation protocol is known as **Hit Ratio @ K**, and it is commonly used to evaluate recommender systems.


<font color='red'>
$TO DO - Students$

- Fill the gaps
  </font>


In [17]:
# Step 2 with hit ratio
def HRatio(test_ratings, predict, K=10):
    # User-item pairs for testing
    test_user_item_set = set(
        list(set(zip(test_ratings["userId"], test_ratings["movieId"])))[:1000]
    )

    # Dict of all items that are interacted with by each user
    user_interacted_items = (
        ratings.groupby("userId")[""" FILL """].apply(list).to_dict()
    )

    hits = []
    for (u, i) in tqdm(test_user_item_set):
        interacted_items = user_interacted_items[u]
        not_interacted_items = set(all_movieIds) - set(interacted_items)
        selected_not_interacted = list(np.random.choice(list(not_interacted_items), 99))
        test_items = selected_not_interacted + [i]
        predicted_labels = predict([""" FILL """, """ FILL """]).reshape(-1)
        topK_items = [test_items[i] for i in np.argsort(predicted_labels)[-K:].tolist()]

        if i in topK_items:
            hits.append(1)
        else:
            hits.append(0)
    hr = np.average(hits)
    print("The Hit Ratio @ {} is {:.2f}".format(K, hr))
    return hr


In [18]:
def predictor(pairs):
    pred = []
    for userId, movieId in zip(pairs[0], pairs[1]):
        pred += [predict(pivot, corr, userId, movieId)]
    return np.array(pred)


HR = dict()
hr = HRatio(test_ratings, predictor, 25)
HR["User based"] = hr


100%|███████████████████████████████████████| 1000/1000 [01:04<00:00, 15.51it/s]

The Hit Ratio @ 25 is 0.71


## Improve the rating


### Trick 1:

Since humans do not usually act the same as critics, i.e., some people usually rank movies higher or lower than others, this prediction function can be easily improved by taking into account the user mean as follows:

$$pred(u, i) = \overline{r_u} + \frac{\sum_v sim(u,v)*(r_{v,i} - \overline{r_v})}{\sum_v sim(u,v)}$$


<font color='red'>
$TO DO - Students$

- Modify the previous code in order to implement "Trick 1"
  </font>


In [ ]:
""" FILL """


### Trick 2:

If two users have very few items in common, let us imagine that there is only one, and the rating is the same, the user similarity will be really high; however, the confidence is really small. It's possible to add a ponderation coefficient.

$$newsim(a, b) = sim(a,b) * \frac{min(N, |P_{a,b}|)}{N}$$

where $|P_{a,b}|$ is the number of common items shared by user a and user b. The coefficient is $< 1$ if the number of common movies is $< N$ and $1$ otherwise.


In [ ]:
# Count the number of common items shared by user 1 and user 2.
pivot = train_pivot.to_numpy()
movie2column = {j: i for i, j in enumerate(train_pivot.columns)}
user2row = {j: i for i, j in enumerate(train_pivot.index)}
a = user2row[1]
b = user2row[2]
common = pivot[a, :] + pivot[b, :]
common = np.count_nonzero(~np.isnan(common))
common


<font color='red'>
$TO DO - Students$

- Modify the previous code in order to implement "Trick 2"
  </font>


In [ ]:
""" FILL """


## To go further

1. Do the same, but with correlation between items. It's Collaborative filtering based on Items similarity. It's also possible to use the 2 previous trick

1. Use Matrix factorization: decompose R in P, Q at rank k (i.e. if R is a m.n matrix, P is a m.k matrix and Q is a n.k matrix) the reconstruct R with P and Q (i.e. $\hat{R} = P Q^T$)

1. Use Matrix decomposition: do an truncated SVD decomposition in order to obtain U, S and V, build $\hat{R} = U S V^T$


<font color='red'>
$TO DO - Students$

- Choose, implement and evaluate one of the above strategies.
  </font>


### Collaborative filtering based on items similarity


In [ ]:
""" TRY to predict with item based collaborative filtering """


In [ ]:
""" Evaluate the result """


### Matrix factorization

Same approach as in the slides.

<font color='blue'>
Below is a simple algorithm for factoring a matrix.
</font>


In [2]:
# Matrix factorization from scratch
def matrix_factorization(R, K, steps=10, alpha=0.005):
    """
    R: rating matrix
    K: latent features
    steps: iterations
    alpha: learning rate
    beta: regularization parameter"""

    # N: num of User
    N = R.shape[0]
    # M: num of Movie
    M = R.shape[1]

    # P: |U| * K (User features matrix)
    P = np.random.rand(N, K)
    # Q: |D| * K (Item features matrix)
    Q = np.random.rand(M, K).T

    for step in tqdm(range(steps)):
        for i in range(N):
            for j in range(M):
                if not np.isnan(R[i][j]):
                    # calculate error
                    eij = R[i][j] - np.dot(P[i, :], Q[:, j])

                    for k in range(K):
                        # calculate gradient with a and beta parameter
                        tmp = P[i][k] + alpha * (2 * eij * Q[k][j])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k])
                        tmp = P[i][k]

    return P, Q.T


In [ ]:
# We try first on a toy example
# R: rating matrix
import math

R = [
    [5, 3, math.nan, 1],
    [4, math.nan, math.nan, 1],
    [1, 1, math.nan, 5],
    [1, math.nan, math.nan, 4],
    [0, 1, 5, 4],
    [2, 1, 3, math.nan],
]

R = np.array(R)
# Num of Features
K = 3

nP, nQ = matrix_factorization(R, K, steps=10)

nR = np.dot(nP, nQ.T)
nR


In [ ]:
""" TRY to predict with matrix factorization """


In [ ]:
""" Evaluate the result """


## Decomposition using latent factor.

We use SVD decomposition


In [ ]:
pivot = train_pivot.fillna(0).to_numpy()
max_components = min(train_pivot.shape) - 1


In [ ]:
from scipy.sparse.linalg import svds

k = 50
assert k < max_components

u, s, v_T = svds(pivot, k=k)
nR = u.dot(np.diag(s).dot(v_T))  # output of TruncatedSVD


In [ ]:
""" TRY to predict with SVD decomposition """


In [ ]:
""" Evaluate the result """
